In [1]:
# install dependencies
!pip install youtube_dl
!pip install transformers
!pip install spacy
!python -m spacy download en_core_web_lg
!pip install sentencepiece
!pip install git+https://github.com/openai/whisper.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 47.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 65.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 70.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 24.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 2.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     

In [4]:
# Shout out to Nabil bd !
# https://www.github.com/Nabilphysics
# https://www.nabilbd.com

import youtube_dl
import os
import whisper
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation
from heapq import nlargest
from transformers import PegasusForConditionalGeneration
from transformers import PegasusTokenizer
from transformers import pipeline
import pandas as pd
# Credit: https://www.activestate.com/blog/how-to-do-text-summarization-with-python/
#punctuation = punctuation + '\n'
#punctuation

In [3]:
#define all functions

# Function to convert youtube link to embed link
# Credit: chatGPT (although chatGPT first mislead me)
def convert_to_embed(link):
    video_id = link.split("=")[-1]
    return 'https://www.youtube.com/embed/' + video_id

def download_ytvid_as_mp3():
    #video_url = input("enter url of youtube video:")
    #https://www.youtube.com/watch?v=6KUn7AKmC0Y
    video_info = youtube_dl.YoutubeDL().extract_info(url = youtube_link,download=False)
    # Your file location for Ubuntu
    myfile = "youtubeVideo.mp3"
   
    # If file exists, delete it.
    if os.path.isfile(myfile):
     os.remove(myfile)
    else:
     # If it fails, inform the user.
     print("Error: %s file not found" % myfile)
    
    #filename = f"{video_info['title']}.mp3"
    filename = f"{'youtubeVideo'}.mp3"
    options={
        'format':'bestaudio/best',
        'keepvideo':False,
        'outtmpl':filename,
    }
    with youtube_dl.YoutubeDL(options) as ydl:
        ydl.download([video_info['webpage_url']])
    print("Download complete... {}".format(filename))


def summarize(text, per):
    nlp = spacy.load('en_core_web_lg')
    doc= nlp(text)
    tokens=[token.text for token in doc]
    word_frequencies={}
    for word in doc:
        if word.text.lower() not in list(STOP_WORDS):
            if word.text.lower() not in punctuation:
                if word.text not in word_frequencies.keys():
                    word_frequencies[word.text] = 1
                else:
                    word_frequencies[word.text] += 1
    max_frequency=max(word_frequencies.values())
    for word in word_frequencies.keys():
        word_frequencies[word]=word_frequencies[word]/max_frequency
    sentence_tokens= [sent for sent in doc.sents]
    sentence_scores = {}
    for sent in sentence_tokens:
        for word in sent:
            if word.text.lower() in word_frequencies.keys():
                if sent not in sentence_scores.keys():                            
                    sentence_scores[sent]=word_frequencies[word.text.lower()]
                else:
                    sentence_scores[sent]+=word_frequencies[word.text.lower()]
    select_length=int(len(sentence_tokens)*per)
    summary=nlargest(select_length, sentence_scores,key=sentence_scores.get)
    final_summary=[word.text for word in summary]
    summary=''.join(final_summary)
    return summary 

In [6]:
df = pd.read_csv('/content/drive/MyDrive/ChatGPTvHumans/YT_Dataset - Sheet1.csv')
df.head()

,Link,Title,Transcript,Summary
0,https://www.youtube.com/watch?v=2-VTLiy0Lls,Can you steal the most powerful wand in the wi...,NaN,NaN
1,https://www.youtube.com/watch?v=KO6bbZOc2B4,How to deal with rejection,NaN,NaN
2,https://www.youtube.com/watch?v=AESkozhaT4s,When are you actually an adult? - Shannon Odell,NaN,NaN
3,https://www.youtube.com/watch?v=BGxYMTftuzw,Mao Zedong's infamous mango cult - Vivian Jiang,NaN,NaN
4,https://www.youtube.com/watch?v=xqaX2yi0VF8,The bizarre world of parasitic wasps - Miles Z...,NaN,NaN


In [17]:
# how to strucuture checkpoint
# for i in range(3,len(df)):
#   print(i)

# how to update rows
# df.loc[2,['Transcript']] = "test!"

#how to write to csv
# df.to_csv('/content/drive/MyDrive/ChatGPTvHumans/YT_Dataset - Sheet1.csv')

In [29]:
# load openAI model whisper
model = whisper.load_model("medium")
# Pick model
model_name = "google/pegasus-xsum"
# Load pretrained tokenizer
pegasus_tokenizer = PegasusTokenizer.from_pretrained('google/pegasus-xsum')
# Define PEGASUS model
pegasus_model = PegasusForConditionalGeneration.from_pretrained(model_name)
# Define summarization pipeline 
summarizer = pipeline(
    "summarization", 
    model=model_name, 
    tokenizer=pegasus_tokenizer, 
    framework="pt"
)

for index in range(0,len(df)):
  
  # read link from dataframe
  youtube_link = df.loc[index,['Link']].Link
  print("Link ",index,"/",len(df)," : " , youtube_link)

  # embed link
  print("\nEmbedding...")
  embed_link = convert_to_embed(youtube_link)
  print(embed_link)

  # download video
  download_ytvid_as_mp3()

  print("\nTranscribing...")
  result = model.transcribe('youtubeVideo.mp3')
  text = result["text"]
  print("Transcription:",text)

  os.remove('youtubeVideo.mp3')

  res = len(text.split())
  # total no of words
  print ("\nThe number of words in string are : " + str(res))

  print("\nConstructive extractive summary...")
  extractive_summary = summarize(text, 0.15) # 1.0 = 100% so 0.15 = 15%
  print(extractive_summary)

  
  # Create tokens
  print("\nCreating Pegasus Tokens...")
  tokens = pegasus_tokenizer(extractive_summary, truncation=True, padding="longest", return_tensors="pt")
  print("Tokens created!")

  print("\nCreating summary...")
  summary = summarizer(extractive_summary, min_length=20, max_length=150)
  print(summary[0]["summary_text"])

  df.loc[index,['Transcript']] = text
  df.loc[index,['Summary']] = summary[0]["summary_text"]

df.to_csv('/content/drive/MyDrive/ChatGPTvHumans/YT_Dataset - Sheet1.csv')

100%|█████████████████████████████████████| 1.42G/1.42G [00:54<00:00, 28.1MiB/s]


Link  0 / 5  :  https://www.youtube.com/watch?v=2-VTLiy0Lls

Embedding...
https://www.youtube.com/embed/2-VTLiy0Lls
[youtube] 2-VTLiy0Lls: Downloading webpage
Error: youtubeVideo.mp3 file not found
[youtube] 2-VTLiy0Lls: Downloading webpage
[download] Destination: youtubeVideo.mp3
[download] 100% of 5.52MiB in 01:07
Download complete... youtubeVideo.mp3

Transcribing...
Transcription:  The fabled Mirzakhani Wand is the most powerful magical item ever created, and that's why the evil wizard Moldavord is planning to use it to conquer the world. You and DrumbleDroar have finally discovered its hiding place in this cave. The wand is hidden by a system of 100 magical stones, including a glowing keystone, and 100 platforms. If the keystone is placed on the correct platform, the wand will be revealed. If placed incorrectly, the entire cave will collapse. The keystone is immune to all magic, but the other stones aren't, meaning you can pick them up and cast a placement spell, and the platform 